# Fashion MNIST — Fast CNN (Wildcard Round)

**Goal:** Train a clean, small CNN on Fashion MNIST with solid documentation and artifacts for submission.

This notebook will:
1. Load & preprocess data
2. Build a compact CNN with Keras
3. Train with callbacks (EarlyStopping + Checkpoint)
4. Evaluate (accuracy, confusion matrix, classification report)
5. Save the model as `fashion_model.keras`

**Tip:** Runtime → Run all. Then download the saved model and generated images.


In [ ]:
# If running on Colab, the below installs are usually not necessary.
# Uncomment if needed.
# !pip -q install -U tensorflow matplotlib scikit-learn
import os, random, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
print(tf.__version__)

In [ ]:
# Reproducibility
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seeds(42)

In [ ]:
# Load data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = (x_train.astype('float32')/255.0)[..., None]
x_test  = (x_test.astype('float32')/255.0)[..., None]

# Create validation split
val_split = 0.1
num_val = int(len(x_train) * val_split)
x_val = x_train[:num_val]
y_val = y_train[:num_val]
x_train_small = x_train[num_val:]
y_train_small = y_train[num_val:]

class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
x_train.shape, x_val.shape, x_test.shape

In [ ]:
# Visualize a few samples
fig = plt.figure(figsize=(6,6))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i].squeeze(), cmap='gray')
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Build model
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name='augmentation')

model = keras.Sequential([
    layers.Input(shape=(28,28,1)),
    data_augmentation,
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint('fashion_model.keras', monitor='val_accuracy', save_best_only=True),
]

history = model.fit(
    x_train_small, y_train_small,
    validation_data=(x_val, y_val),
    epochs=12, batch_size=128, verbose=2,
    callbacks=callbacks,
)
with open('training_history.json', 'w') as f:
    json.dump(history.history, f)

print('Best model saved to fashion_model.keras')

In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc:.4f}  |  Test loss: {test_loss:.4f}')

# Predictions & reports
y_pred = np.argmax(model.predict(x_test, verbose=0), axis=1)
report = classification_report(y_test, y_pred, target_names=class_names, digits=4)
print(report)
with open('classification_report.txt', 'w') as f:
    f.write(report)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, normalize='true')
fig = plt.figure(figsize=(6,6))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix (normalized)')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45, ha='right')
plt.yticks(tick_marks, class_names)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
fig.savefig('confusion_matrix.png', bbox_inches='tight')
plt.show()

In [ ]:
# Sample predictions visualization
idxs = np.random.choice(len(x_test), size=9, replace=False)
fig = plt.figure(figsize=(6,6))
for i, idx in enumerate(idxs):
    img = x_test[idx]
    pred = y_pred[idx]
    true = y_test[idx]
    plt.subplot(3,3,i+1)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(f'P: {class_names[pred]}\nT: {class_names[true]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

## ✅ Next Steps (for submission)
- Download `fashion_model.keras` and include it in your ZIP.
- Add `classification_report.txt` and `confusion_matrix.png` to your ZIP (optional but nice).
- Fill in your metrics in `REPORT.md`.
- Zip everything as `YourName_ML.zip` and submit.
